In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load and preprocess OI data
oi_df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/YBA OI.csv", skiprows=[0])
oi_df.rename(columns={oi_df.columns[0]: "Date", oi_df.columns[1]: "Open Interest"}, inplace=True)
oi_df["Date"] = pd.to_datetime(oi_df["Date"], errors='coerce')
oi_df = oi_df.dropna(subset=["Date", "Open Interest"])
oi_df = oi_df.set_index("Date").resample("W").last().dropna().reset_index()

# Load and preprocess Volume data
pv_df = pd.read_csv("C:/Users/jay.jain/Desktop/Codes - Jay/Assignment 1/YBAC1.csv")
pv_df["Date"] = pd.to_datetime(pv_df["Date-Time"], errors='coerce').dt.tz_localize(None)
pv_df = pv_df.dropna(subset=["Date", "Volume"])
pv_df = pv_df.set_index("Date").resample("W").sum().reset_index()

# Merge OI and Volume
merged = pd.merge(oi_df, pv_df, on="Date", how="inner")
merged = merged[(merged["Date"] >= "2012-01-01") & (merged["Date"] <= "2015-12-31")]
merged["Day"] = merged["Date"].dt.day
merged["Month"] = merged["Date"].dt.month

# Pivot table for heatmap
heatmap_data = merged.pivot_table(index="Day", columns="Month", values="Open Interest", aggfunc="mean")

# Clip extreme values to reduce skew in visualization
vmin, vmax = np.percentile(heatmap_data.values.flatten(), [5, 95])
heatmap_data_clipped = heatmap_data.clip(lower=vmin, upper=vmax)

# Plot
plt.figure(figsize=(12, 8))
sns.heatmap(
    heatmap_data_clipped,
    cmap="YlGnBu",
    linewidths=0.5,
    linecolor='gray',
    cbar_kws={'label': 'Avg Open Interest'}
)
plt.title("Heatmap of Avg Weekly Open Interest (2012–2015)")
plt.xlabel("Month")
plt.ylabel("Day of Month")
plt.tight_layout()
plt.show()